In [1]:
import pandas as pd

#데이터 불러오기
original_data = pd.read_csv("data.csv", encoding='cp949')
data = original_data

### 1. 데이터 확인

In [2]:
data.head()

,도서관명,시도명,시군구명,도서관유형,휴관일,평일운영시작시각,평일운영종료시각,토요일운영시작시각,토요일운영종료시각,공휴일운영시작시각,...,운영기관명,도서관전화번호,부지면적,건물면적,홈페이지주소,위도,경도,데이터기준일자,제공기관코드,제공기관명
0,이석영뉴미디어도서관,경기도,남양주시,공공도서관,둘째·넷째 금요일+1월 1일+설날·추석 연휴(대체공휴일 포함),9:00,22:00,9:00,18:00,9:00,...,사단법인 문화현상,031-595-8390,1843.0,4877.0,http://lib.nyj.go.kr/lsy,37.658160,127.299532,2023-04-30,3990000,경기도 남양주시
1,원덕도서관,강원도,삼척시,공공도서관,토요일+일요일+법정공휴일,9:00,18:00,0:00,0:00,0:00,...,강원도 삼척시청,033-572-4819,766.0,364.0,NaN,37.176854,129.335484,2023-06-20,4241000,강원특별자치도 삼척시
2,가평군 한석봉도서관,경기도,가평군,공공도서관,1월 1일+설 연휴+추석연휴,7:00,23:59,7:00,23:59,7:00,...,경기도 가평군,031-580-4041,7485.0,3602.0,www.gaplib.go.kr,37.829691,127.506433,2023-06-21,4160000,경기도 가평군
3,가평군 설악도서관,경기도,가평군,공공도서관,1월 1일+설 연휴+추석연휴,7:00,23:59,7:00,23:59,7:00,...,경기도 가평군,031-580-4313,3693.0,1470.0,www.gaplib.go.kr,37.683315,127.491015,2023-06-21,4160000,경기도 가평군
4,가평군 청평도서관,경기도,가평군,공공도서관,1월 1일+설 연휴+추석연휴,7:00,23:59,7:00,23:59,7:00,...,경기도 가평군,031-580-4332,4500.0,2486.0,www.gaplib.go.kr,37.739591,127.424230,2023-06-21,4160000,경기도 가평군


In [3]:
#NaN값 확인 - 위도, 경도가 없는 경우가 많음 -> 소재지 도로명 주소는 있음
data.isnull().sum()

도서관명             0
시도명              0
시군구명             0
도서관유형            0
휴관일              0
평일운영시작시각         0
평일운영종료시각         0
토요일운영시작시각        0
토요일운영종료시각        0
공휴일운영시작시각        0
공휴일운영종료시각        0
열람좌석수            0
자료수(도서)          0
자료수(연속간행물)       0
자료수(비도서)         0
대출가능권수           0
대출가능일수           0
소재지도로명주소         0
운영기관명          668
도서관전화번호        233
부지면적          1610
건물면적           481
홈페이지주소        1422
위도             258
경도             247
데이터기준일자          0
제공기관코드           0
제공기관명            0
dtype: int64

In [4]:
data1 = data.loc[:, ['도서관명', '소재지도로명주소', '위도', '경도']]

In [5]:
# 위도, 경도를 도로명주소로 찾아서 변환
# 위도나 경도가 NaN인 값을 찾아서 데이터프레임 출력
nan_df = data1[data['위도'].isnull() | data['경도'].isnull()]

print(nan_df.count())  # 258

nan_df.head()

도서관명        258
소재지도로명주소    258
위도            0
경도           11
dtype: int64


,도서관명,소재지도로명주소,위도,경도
45,사천시어린이도서관,경상남도 사천시 사남면 조동길 49-30,NaN,NaN
78,맑은숲작은도서관,울산광역시 동구 명덕6길 29,NaN,NaN
161,크레파스작은도서관,울산광역시 동구 동해안로 98,NaN,NaN
262,손기정문화도서관,서울특별시 중구 손기정로101,NaN,NaN
263,장충동 작은도서관,서울특별시 중구 동호로25길 33,NaN,NaN


### 2. 전처리

#### 2-1. 위경도가 없는 데이터 채워주기

In [6]:
# 위경도 변환하기 위한 라이브러리 설치
!pip install geopy

In [7]:
# 위경도 변환 라이브러리 불러오기
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="library_map")

#### 2-1-1. 위경도 찾기 위해 주소 가공 - 정규표현식 사용

In [74]:
# 정규표현식 사용 - 위경도 찾기 위한 주소 가공
import re

'''
1. 뒤에 있는 괄호 지우기        / (지족동) -> ""
2. (숫자)번길 앞에 띄어쓰기를 지운다.       / 수레로 1260번길 -> 수레로1260번길
3. 번길 뒤에 숫자가 있을 경우 띄어쓰기를 한다.  / 1260번길33 -> 1260번길 33

** 추가 ***********
o4. 숫자 앞에 띄어쓰기 없으면 하기   / 대학길40-1 -> 대학길 40-1
5. 쉼표(,) 붙은거 뒷부분 다 지우기
6. (숫자)층, (숫자)동 붙은거 뒷부분은 다 지우기
7. 앞뒤에 strip()함수 써서 띄어쓰기 제거하기
********************

'''

def address_process(text):


    # 1. 뒤에 있는 괄호 지우기
    round = re.search('\(.*\)$', text)
    if round:
        round = round.group()
        text = re.sub('\(.*\)$', '', text)

    # 2. '(숫자)번길' 앞에 띄어쓰기를 지운다.
    road = re.search('\s*\d+번길', text)
    if road:
        road = road.group().strip()
        text = re.sub('\s*\d+번길', road, text)  # 경기도 남양주시 화도읍 수레로1260번길33
        #print(road)

    # 3. '번길' 뒤에 숫자가 있을 경우 띄어쓰기를 한다.
    num = re.search('\d+번길\d+', text)
    if num:
        num = num.group()
        num = re.search('\d+$', num).group()
        #print(num)  #33

    # 4. 숫자 앞에 띄어쓰기 없으면 하기
    # 숫자 앞에 번길이 없는 경우에만 할 것
    # s*\d+-*\d+ 숫자는 40-1 과 401 둘다 찾는다.
    if not num:
        num_space = re.search('s*\d+-*\d+', text)
        if num_space:
            num_space = num_space.group()
            text = re.sub('s*\d+-*\d+',' '+ num_space, text)




    if road and num:
        result = re.sub('\d+번길\d+', road+" "+num, text)  # 경기도 남양주시 화도읍 수레로 1260번길 33
    elif road:
        result = re.sub('\d+번길\d+', road, text)
    elif num:
        result = re.sub('\d+번길\d+', num, text)
    else:
        result = text

    #print(result)
    return result

In [75]:
text = "울산광역시 동구 대학길40-1"
address_process(text)

'울산광역시 동구 대학길 40-1'

In [56]:
# 위경도 변환 샘플 예제
location = geolocator.geocode("경기도 여주시 흥천면 이여로 1278-2")
print(location)
if location:
    print(location.latitude)
    print(location.longitude)

    print((location.latitude, location.longitude))

None


In [64]:
# 위경도 변환 샘플 예제
location = geolocator.geocode("인천광역시 남동구 정각로 9")
print(location)
if location:
    print(location.latitude)
    print(location.longitude)

    print((location.latitude, location.longitude))

정각로, 인천광역시교육청, 구월1동, 남동구, 인천, 21554, 대한민국
37.4551861
126.7032169
(37.4551861, 126.7032169)


In [10]:
# 위경도 변환 함수
def coord(address):

    result = ""
    location = geolocator.geocode(address)
    #print(location)
    if location:
        #print(location.latitude)
        #print(location.longitude)
        result = (location.latitude, location.longitude)

    return result

In [11]:
# 테스트를 위하여 위도와 경도가 NaN를 가진
# 데이터들 중에 소재지도로명주소 열만 가져옴
nan_address = nan_df.loc[:,'소재지도로명주소']

In [12]:
# 위경도를 구해서 result_list 리스트에 담음
result_list = []
for e in nan_address:
    address = address_process(e)
    #print(address)
    result = coord(address)

    result_list.append(result)

print(result_list)

['', (35.5241701, 129.4317128), (35.5429174, 129.4303354), '', (37.5587151, 127.0015589), (37.5498323, 127.0058824), (37.07417, 127.38598), '', '', '', (35.508852700000006, 129.42671760151546), (35.50490275, 129.41665009077644), (35.50316, 129.42927), '', (35.484532, 129.4158766), (35.4909863, 129.41860598893481), (35.52281, 129.42899), (35.5166998, 129.4306185), (35.5434089, 129.4275833), (35.4861154, 129.4157592), (35.4899476, 129.4262602134266), (35.5137825, 129.4289254), '', '', (37.0053735, 127.2593355), '', (37.0079107, 127.1842449), (37.0043472, 127.1703209), (36.9997577, 127.154667), '', '', (35.1831242, 128.1030948), '', (35.7013006, 127.6863483), (35.4807762, 127.6845917), '', '', '', '', '', '', (35.5517421, 127.6368805), '', '', '', '', '', (35.9414261, 128.9001449), '', (34.9634415, 127.7163532), (34.5984306, 127.2766145), (34.947793000000004, 127.69091310095111), '', (34.798281, 126.6947291), '', '', '', '', '', '', '', '', '', '', '', '', '', '', (35.275269, 127.1350606)

In [13]:
print(len(nan_df))      # 위경도가 없는 데이터 개수
print(len(result_list))
print(result_list.count(''))  # 위경도를 찾지 못한 데이터 - 97

258
258
97


In [14]:
# 위경도를 찾은 데이터는 df에 넣기
import numpy as np

result_array = np.array(result_list)  #list타입을 array로 바꿔준다.
not_nan_index = np.where(result_array != "")[0]
#print(index)

#nan가 아닌 데이터의 인덱스 번호 구하기
not_nan_index = nan_df.index[not_nan_index]
#print(not_nan_index)

<ipython-input-14-a33576d4aed5>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result_array = np.array(result_list)  #list타입을 array로 바꿔준다.


In [15]:
# 값이 있는 result_list만 사용
add_coord = [x for x in result_list if x != '']
len(add_coord)

161

#### 2-1-2. 위경도 찾은 것을 original_data에 값 넣어주기

In [16]:
#not_nan_index 번호를 for문 돌리면서 거기에 해당하는 위경도 좌표를 넣어줌
for i in range(len(not_nan_index)):
    index = not_nan_index[i]

    original_data.loc[index, '위도'] = add_coord[i][0]
    original_data.loc[index, '경도'] = add_coord[i][1]

In [17]:
nan_df2 = data1[original_data['위도'].isnull() | original_data['경도'].isnull()]
print(len(nan_df2)) #기존 258 -> 97 로 줄어듬

97


============================

#### 2-1-3. 위의 방법으로 위경도를 찾지 못한 데이터 처리

In [19]:
# result_list에서 비어있는 값을 가진 인덱스 번호 찾기
import numpy as np
result_array = np.array(result_list)  #list타입을 array로 바꿔준다.
nan_index = np.where(result_array == "")[0]
print(nan_index)

[  0   3   7   8   9  13  22  23  25  29  30  32  35  36  37  38  39  40
  42  43  44  45  46  48  52  54  55  56  57  58  59  60  61  62  63  64
  65  66  67  69  73  74  75  76  77  78  79  80  81  92  95  96  97  98
 101 105 112 113 120 122 123 124 126 127 129 130 131 132 146 149 160 161
 162 163 164 165 166 168 171 172 173 174 175 176 177 185 189 190 205 215
 217 221 225 252 253 254 255]


<ipython-input-19-096fb95d882b>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result_array = np.array(result_list)  #list타입을 array로 바꿔준다.


In [20]:
print(len(nan_index))  #97

97


In [29]:
nan_index

Int64Index([  45,  262,  341,  342,  343,  349,  398,  399,  401,  405,  459,
             498,  501,  502,  503,  526,  542,  543,  545,  546,  587,  588,
             589,  641,  688,  804,  805,  806,  807,  808,  809,  811,  812,
             815,  816,  817,  818,  819,  821,  906, 1036, 1037, 1038, 1039,
            1040, 1041, 1042, 1043, 1044, 1705, 1750, 1778, 1820, 1841, 1848,
            1917, 2072, 2073, 2310, 2312, 2313, 2314, 2316, 2317, 2319, 2420,
            2421, 2428, 2655, 2658, 3020, 3021, 3022, 3023, 3072, 3073, 3074,
            3198, 3242, 3243, 3244, 3245, 3246, 3247, 3248, 3375, 3379, 3391,
            3573, 3583, 3585, 3589, 3593, 3620, 3621, 3622, 3623],
           dtype='int64')

In [31]:
# 위경도가 nan 값인 것 찾기
#nan_df2  # 97
#nan_data = original_data.loc[nan_index] # 97

In [34]:
import pandas as pd
nan_df2.to_excel('nan_data.xlsx') #nan 데이터 엑셀로 확인

====================================================

### 테스트

In [ ]:
dic = {"위도" : data1['위도'], "경도" : data1['경도']}

In [ ]:
location1 = pd.DataFrame(dic)

In [ ]:
location1

,위도,경도
0,37.658160,127.299532
1,37.176854,129.335484
2,37.829691,127.506433
3,37.683315,127.491015
4,37.739591,127.424230
...,...,...
3624,NaN,NaN
3625,NaN,NaN
3626,38.125564,128.205558
3627,37.955236,128.319014


In [ ]:
for i in location1:
    print(i)
    break

0       37.658160
1       37.176854
2       37.829691
3       37.683315
4       37.739591
          ...    
3624          NaN
3625          NaN
3626    38.125564
3627    37.955236
3628    38.221730
Name: 위도, Length: 3629, dtype: float64


In [ ]:
# 지도 시각화를 위해 라이브러리 불러옴
import folium

#folium.Map(location=[위도, 경도])
m = folium.Map(location=[37.658160, 127.299532])

In [ ]:
folium.Marker([37.658160, 127.299532]).add_to(m)
m